In [1]:
import torch
from transformers import AutoTokenizer, PhimoeConfig, PhimoeForCausalLM
from transformers.utils import logging
from accelerate import init_empty_weights, infer_auto_device_map


logging.set_verbosity_info()
device = torch.device("cuda:0")
# Model path
#"microsoft/Phi-tiny-MoE-instruct"
#"microsoft/Phi-3.5-MoE-instruct"
model_path = "microsoft/Phi-tiny-MoE-instruct"

# Load config
config = PhimoeConfig.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
with init_empty_weights():
    model = PhimoeForCausalLM(config)

model = PhimoeForCausalLM.from_pretrained(model_path,
                                        device_map="auto",
                                        torch_dtype=torch.bfloat16,
                                    )
model.eval()


🚨 `use_probabilistic_routing` is part of PhimoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in X:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\transformers\src\transformers\models\phimoe\modeling_phimoe.py.
🚨 `prob_routing_temp` is part of PhimoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in X:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\transformers\src\transformers\models\phimoe\modeling_phimoe.py.
🚨 `use_probabilistic_routing` is part of PhimoeForCausalLM.forward's signature, but not documented. Make sure to add it to the docstring of the function in X:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\transformers\src\transformers\models\phimoe\modeling_phimoe.py.
🚨 `prob_routing_temp` is part of PhimoeForCausalLM.forward's signature, 

config.json: 0.00B [00:00, ?B/s]

x:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\LLM-Eng-Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Skale\.cache\huggingface\hub\models--microsoft--Phi-tiny-MoE-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
loading configurati

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Instantiating PhimoeForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Error(s) in loading state_dict for Linear:
	size mismatch for bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([1024]).

In [3]:
if "inputs" in globals():
    del inputs

prompt = 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.' +' Is the prior text positive or negative? Give us a rating out of 10'
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# prob_routing_temp cannot be lower than about 0.0001. 
# 0.00001 causes cuda assert


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        use_probabilistic_routing=True,
        prob_routing_temp=0.5,
        #output_router_logits=True,
        #return_dict=True
    )

print(tokenizer.decode(outputs[0]))

tensor([[ 3,  5],
        [ 5,  3],
        [12, 13],
        [11, 15],
        [ 8, 11],
        [12, 15],
        [ 3,  8],
        [10, 15],
        [ 8,  1],
        [ 8, 11],
        [ 5, 11],
        [ 6, 15],
        [ 8, 11],
        [11,  8],
        [ 5, 12],
        [15,  9],
        [ 8, 11],
        [14,  5],
        [11, 13],
        [11, 12],
        [ 8,  1],
        [ 3, 11],
        [ 5,  4],
        [12, 10],
        [ 4, 11],
        [ 5,  3],
        [14,  6],
        [ 7, 15],
        [ 8, 11],
        [ 5, 11],
        [ 5, 14],
        [ 6, 15],
        [ 8, 11],
        [ 1, 15],
        [ 8, 11],
        [ 3, 11],
        [ 4,  8],
        [14,  5],
        [ 4, 11],
        [ 7, 15],
        [ 8, 11],
        [ 1, 15],
        [ 1, 15],
        [ 3, 11],
        [ 4, 12],
        [ 8, 11],
        [ 8, 15],
        [ 4, 11],
        [11, 13],
        [14,  4],
        [ 8, 12],
        [ 4, 11],
        [10, 15],
        [ 1, 15],
        [11,  8],
        [ 

In [4]:
if "inputs" in globals():
    del inputs

prompt = 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.' +' Is the prior text positive or negative? Give us a rating out of 10'
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# prob_routing_temp cannot be lower than about 0.0001. 
# 0.00001 causes cuda assert


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        use_probabilistic_routing=False,
        prob_routing_temp=0.01,
        #output_router_logits=True,
        #return_dict=True
    )

print(tokenizer.decode(outputs[0]))

tensor([[ 3,  5],
        [ 5,  3],
        [12, 13],
        [11, 15],
        [ 8, 11],
        [12, 15],
        [ 3,  8],
        [10, 15],
        [ 8,  1],
        [ 8, 11],
        [ 5, 11],
        [ 6, 15],
        [ 8, 11],
        [11,  8],
        [ 5, 12],
        [15,  9],
        [ 8, 11],
        [14,  5],
        [11, 13],
        [11, 12],
        [ 8,  1],
        [ 3, 11],
        [ 5,  4],
        [12, 10],
        [ 4, 11],
        [ 5,  3],
        [14,  6],
        [ 7, 15],
        [ 8, 11],
        [ 5, 11],
        [ 5, 14],
        [ 6, 15],
        [ 8, 11],
        [ 1, 15],
        [ 8, 11],
        [ 3, 11],
        [ 4,  8],
        [14,  5],
        [ 4, 11],
        [ 7, 15],
        [ 8, 11],
        [ 1, 15],
        [ 1, 15],
        [ 3, 11],
        [ 4, 12],
        [ 8, 11],
        [ 8, 15],
        [ 4, 11],
        [11, 13],
        [14,  4],
        [ 8, 12],
        [ 4, 11],
        [10, 15],
        [ 1, 15],
        [11,  8],
        [ 

NameError: name 'model' is not defined